In [0]:
ROOT='/content/drive/My Drive/dl/推荐评论/'
BERT_ROOT='/content/drive/My Drive/dl/google-quest-challenge/pretrained-bert-models-for-pytorch/新建文件夹/bert-base-chinese/'
BERT_TOK_ROOT='/content/drive/My Drive/dl/google-quest-challenge/pretrained-bert-models-for-pytorch/bert-base-chinese-vocab.txt'
MAX_LEN=22

In [2]:
!pip install pytorch_pretrained_bert
!pip install transformers
!pip install scikit-learn==0.20.2

     |████████████████████████████████| 133kB 24.6MB/s 
     |████████████████████████████████| 501kB 35.9MB/s 
     |████████████████████████████████| 1.0MB 48.2MB/s 
     |████████████████████████████████| 870kB 52.7MB/s 
     |████████████████████████████████| 3.7MB 52.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=e66081947d3d76a9031120d99e9b96e8f83f1c765318d6da3f156c96ee6fb85c
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses
     |████████████████████████████████| 5.4MB 15.5MB/s 
  Found existing installation: scikit-learn 0.22.1
    Uninstalling scikit-learn-0.22.1:
      Successfully uninstalled scikit-learn-0.22.1


In [3]:
import sklearn
import torch
import pandas as pd
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from pytorch_pretrained_bert import BertModel, BertTokenizer
root='/content/drive/My Drive/google-quest-challenge/'
import numpy as np
import time
import tqdm

import math
import scipy
import torch.nn.functional as F
import random
import transformers, sys, os, gc
from transformers import (
    BertTokenizer, BertModel, BertForSequenceClassification, BertConfig,
    WEIGHTS_NAME, CONFIG_NAME, AdamW, get_linear_schedule_with_warmup, 
    get_cosine_schedule_with_warmup,
)
from torch.utils.data import DataLoader, Dataset,RandomSampler, SequentialSampler
from transformers.modeling_bert import BertPreTrainedModel 
from sklearn.model_selection import StratifiedShuffleSplit,StratifiedKFold
from tqdm import tqdm_notebook
from sklearn.metrics import roc_curve 
import numpy as np

In [0]:
def setup_seed(seed):
     torch.manual_seed(seed)
     torch.cuda.manual_seed_all(seed)
     np.random.seed(seed)
     random.seed(seed)
     torch.backends.cudnn.deterministic = True
setup_seed(42)

In [0]:

test=[]
weight=[]

In [0]:
with open (ROOT+'data/test.txt', 'r') as f:
  for line in f:
    text = line
    test.append(text.replace('\n',''))

In [7]:
tokenizer=BertTokenizer.from_pretrained(BERT_TOK_ROOT)
test_idx = []
test_mask = []
test_segment = []
lenth = []

Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated


In [0]:
for line in range(len(test)):
  token = tokenizer.tokenize(test[line])
  token = ["[CLS]"] + token + ["[SEP]"]
  if len(token)> MAX_LEN:
      raise IndexError("Token length more than max seq length!")
  lenth.append(len(token))
  mask = [1]*len(token)+ [0]*(MAX_LEN-len(token))
  sent = [0]*MAX_LEN
  idx = tokenizer.convert_tokens_to_ids(token)
  idx = idx + (MAX_LEN-len(idx))*[0]
  test_idx.append(idx)
  test_mask.append(mask)
  test_segment.append(sent)

In [0]:
lenths = np.array(lenth)
lenths=lenths.reshape(-1,1)
use = lenths
use_shape = use.shape[1]
use = list(use)

In [0]:
class EvalDataset(Dataset):
  def __init__(self,train_idx,train_mask,train_segment,use,label=None):
    self.train_idx = torch.tensor(train_idx,dtype=torch.int).long()
    self.train_mask = torch.tensor(train_mask,dtype=torch.int).long()
    self.train_segment = torch.tensor(train_segment,dtype=torch.int).long()
    self.use = torch.tensor(use , dtype=torch.float32)
    self.label = None
    if label is not None:
      self.label = torch.tensor(label,dtype=torch.float32)
  def __len__(self):
    return len(self.train_idx)
  def __getitem__(self,item):
    if self.label is not None:

      output = {
        'train_idx' : self.train_idx[item],
        'train_mask' : self.train_mask[item],
        'train_segment' :self.train_segment[item],
        'use':self.use[item],
        'label' : self.label[item]
    }
    else:
      output = {
        'train_idx' : self.train_idx[item],
        'train_mask' : self.train_mask[item],
        'use' : self.use[item],
        'train_segment' :self.train_segment[item],
      }
    return {key: value for key, value in output.items()}

In [0]:
class CLSbert(BertPreTrainedModel):
  def __init__(self,config,use_shape):
    super(CLSbert,self).__init__(config)
    self.bert = BertModel(config)
    self.dropout = nn.Dropout(0.3)
    
    self.softmax = nn.Softmax(dim=1)
    self.pre = nn.Linear(2*768+use_shape,1)
    self.linear1 = nn.Linear(2*768+use_shape,2*768+use_shape)
    self.dropoutlist = nn.ModuleList([nn.Dropout(0.7) for _ in range(7)])

    self.ln = nn.LayerNorm(2*768+use_shape)
    self.d_model = 768
    self.wt=torch.nn.Parameter(torch.rand((13,1),dtype=torch.float32,device=torch.device('cuda'),requires_grad=True))
    nn.init.xavier_normal_(self.wt)
  def forward(
        self,
        use,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
    ):
    outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )
    w=outputs[2] 

    batch_size1,len_seq1,e_s1=w[1].size()
    bert_output1 = w[-1][0][int(use[0].item())-1].unsqueeze(0) #0个batch
    for i in range(1,batch_size1):
        bert_output1=torch.cat((bert_output1,w[-1][i][int(use[0].item())-1].unsqueeze(0)),dim=0)
    bert_output=w[0][:,0].reshape((-1,1,self.d_model)).to(device).clone()
    for i in range(1,13):
        bert_output=torch.cat((bert_output,w[i][:,0].reshape((-1,1,self.d_model)).to(device)),dim=1)
    bert_output=bert_output.view(batch_size1,13,-1)
    bert_output=bert_output.permute(0,2,1)
    bert_output=torch.matmul(bert_output,self.softmax(self.dropout(self.wt)))
    bert_output=torch.cat((bert_output.view(batch_size1,e_s1),use),dim=-1)
    bert_output=torch.cat((bert_output,bert_output1),dim=-1)
    l1=(self.linear1((self.dropoutlist[0]((bert_output)))))
    l2=(self.linear1((self.dropoutlist[1]((bert_output)))))
    l3=(self.linear1((self.dropoutlist[2]((bert_output)))))
    l4=(self.linear1((self.dropoutlist[3]((bert_output)))))
    l5=(self.linear1((self.dropoutlist[4]((bert_output)))))
    l6=(self.linear1((self.dropoutlist[5]((bert_output)))))
    l7=(self.linear1((self.dropoutlist[6]((bert_output)))))
    l=(l1+l2+l3+l4+l5+l6+l7)/7

    logits = self.pre(l)
    return logits

In [0]:

## Stolen from transformer code base without any noble intention.

## Stolen from transformer code base without any noble intention.
class Attention(nn.Module):
    def __init__(self, feature_dim, step_dim, bias=True, **kwargs):
        super(Attention, self).__init__(**kwargs)
        
        self.supports_masking = True

        self.bias = bias
        self.feature_dim = feature_dim
        self.step_dim = step_dim
        self.features_dim = 0
        
        weight = torch.zeros(feature_dim, 1)
        nn.init.xavier_uniform_(weight)
        self.weight = nn.Parameter(weight)
        
        if bias:
            self.b = nn.Parameter(torch.zeros(step_dim))
        
    def forward(self, x, mask=None):
        feature_dim = self.feature_dim
        step_dim = self.step_dim

        eij = torch.mm(
            x.contiguous().view(-1, feature_dim), 
            self.weight
        ).view(-1, step_dim)
        
        if self.bias:
            eij = eij + self.b
            
        eij = torch.tanh(eij)
        a = torch.exp(eij)
        
        if mask is not None:
            a = a * mask

        a = a / torch.sum(a, 1, keepdim=True) + 1e-10
        
        weighted_input = x.permute(1,0,2) * torch.unsqueeze(a, -1)
        return torch.sum(weighted_input, 1)
class cnnbert(BertPreTrainedModel):

    def __init__(self, config,kernel_sizes,num_channels,use_shape):
        super(cnnbert, self).__init__(config)
        self.num_labels = config.num_labels
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(0.3)
        self.bn = nn.LayerNorm(768)
        self.gelu=GELU()
        self.att1=Attention(512,21)
        self.att2=Attention(512,20)
        self.att3=Attention(512,19)
        self.att4=Attention(512,18)
        self.att5=Attention(512,13)
        self.softmax=nn.Softmax(dim=1)
        self.pre_cnn=nn.Linear(2*sum(num_channels)+use_shape,1)
        self.linear = nn.Linear(2*sum(num_channels)+use_shape,2*sum(num_channels)+use_shape)

        self.ln_cnn=nn.LayerNorm(sum(num_channels))
        self.wt=torch.nn.Parameter(torch.rand((13,1),dtype=torch.float32,device=torch.device('cuda'),requires_grad=True))
        nn.init.xavier_normal_(self.wt)
        self.pool = GlobalAVGPool1d()
        self.convs = nn.ModuleList()
        for c, k in zip(num_channels, kernel_sizes):
          self.convs.append(nn.Conv1d(in_channels = 768,
          out_channels = c,
          kernel_size = k))

    def forward(
        self,
        use,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        
    ):

        outputs = self.bert(
           
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )

        w1=outputs[2] 
        batch_size1,len_seq1,e_s1=w1[1].size()
        bert_output1=w1[0].to(device)
        bert_output1=bert_output1.unsqueeze(1)
        for i in range(1,13):
          bert_output1=torch.cat((bert_output1,w1[i].to(device).unsqueeze(1)),dim=1)
        bert_output1=bert_output1.view(batch_size1,13,-1)
        bert_output1=bert_output1.permute(0,2,1)
        bert_output1=torch.matmul(bert_output1,((self.wt)))
        bert_output1=bert_output1.view(batch_size1,len_seq1,e_s1)#下可接CNN或者att
        bert_output1=self.bn(bert_output1)
        bert_output1=bert_output1.permute(0, 2, 1)

        bert_output1 = self.dropout(bert_output1)
        conout1=self.att1(self.gelu(self.convs[0](bert_output1).permute(2,0,1)))
        conout2=self.att2(self.gelu(self.convs[1](bert_output1).permute(2,0,1)))
        conout3=self.att3(self.gelu(self.convs[2](bert_output1).permute(2,0,1)))
        conout4=self.att4(self.gelu(self.convs[3](bert_output1).permute(2,0,1)))
        conout5=self.att5(self.gelu(self.convs[4](bert_output1).permute(2,0,1)))
        conout=torch.cat((conout1,conout2,conout3,conout4,conout5),dim=1)
        encoding =torch.cat([self.pool(self.gelu(conv(bert_output1))).squeeze(-1) for conv in self.convs], dim=1)
       
        #lin_output = F.relu(self.bn(encoding)) ## Note : This Linear layer is added without expert supervision . This will worsen the results . 
        encoding=torch.cat((encoding,conout,use),dim=1)
        #encoding =torch.cat([self.pool(self.gelu(conv(bert_output1))).squeeze(-1) for conv in self.convs], dim=1)
        #lin_output = F.relu(self.bn(encoding)) ## Note : This Linear layer is added without expert supervision . This will worsen the results . 
        #lin_output = F.relu(self.bn(encoding)) ## Note : This Linear layer is added without expert supervision . This will worsen the results .                                     ## But you are smarter than me , so you will figure out,how to customize better.
        logits = self.pre_cnn((encoding))
        return logits
class GlobalAVGPool1d(nn.Module):
    def __init__(self):
        super(GlobalAVGPool1d, self).__init__()
    def forward(self, x):
         # x shape: (batch_size, channel, seq_len)
        return F.avg_pool1d(x, kernel_size=x.shape[2]) # shape: (batch_size, channel, 1) 

In [0]:
class GELU(nn.Module):
    """
    Paper Section 3.4, last paragraph notice that BERT used the GELU instead of RELU
    """

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))

In [0]:
class EMA():
    def __init__(self, model, decay):
        self.model = model
        self.decay = decay
        self.shadow = {}
        self.backup = {}

    def register(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                self.shadow[name] = param.data.clone()

    def update(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                new_average = (1.0 - self.decay) * param.data + self.decay * self.shadow[name]
                self.shadow[name] = new_average.clone()

    def apply_shadow(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                self.backup[name] = param.data
                param.data = self.shadow[name]

    def restore(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.backup
                param.data = self.backup[name]
        self.backup = {}

In [0]:
def predict_model(model,predict_loader ,pre_shape,batch_size , ema, config=None):
  #ema.apply_shadow()
  model.eval()
  acc_lm = 0
  acc_sent = 0
  avg_loss = 0
  pred = np.zeros((pre_shape))
  with torch.no_grad():
    for idx, batch in tqdm_notebook(enumerate(predict_loader),mininterval=2,desc='--Valing',leave=False):
      input_ids = batch['train_idx'].to(device)
      input_mask = batch['train_mask'].to(device)
      input_segments = batch['train_segment'].to(device)
      use = batch['use'].to(device)
      output_train = model(use=use,input_ids = input_ids.long(),
                              labels = None,
                              attention_mask = input_mask,
                              token_type_ids = input_segments,)
      pred[idx*batch_size : (idx+1)*batch_size]=(torch.sigmoid(output_train.cpu().view(-1)))
  #ema.restore()
  return pred

In [0]:
bert_model_config = BERT_ROOT+'/bert_config.json'
bert_config = BertConfig.from_json_file(bert_model_config)
bert_config.num_labels = 1
bert_config.output_hidden_states=True


In [0]:
dataset = EvalDataset(test_idx,test_mask,test_segment,use)
pre_shape = len(test_idx)
pre_loader = DataLoader(dataset,batch_size=128,shuffle=False)
pred = np.zeros((pre_shape))


In [0]:
device = torch.device('cuda')

In [21]:
len(test_idx)

4189

In [23]:
for i in [1,2,3,5]:
  kernel_sizes,nums_channels = [2,3,4,5,10], [512,512,512,512,512]
  model = cnnbert.from_pretrained(BERT_ROOT,kernel_sizes=kernel_sizes,num_channels=nums_channels,use_shape=use_shape,config = bert_config)
  model.to(device)
  #model.load_state_dict(torch.load('/content/drive/My Drive/dl/推荐评论/CNN/best_param_score_{}.pkl'.format(i)))
  ema=torch.load('/content/drive/My Drive/dl/推荐评论/0.96840版本基础 伪标签/CNN/EMA{}.pkl'.format(i))
  ema.model = model
  ema.apply_shadow()
  pred += predict_model(model,pre_loader ,pre_shape,128, ema,config=None)

In [0]:
for i in [1,2,3,5]:
  model = CLSbert.from_pretrained(BERT_ROOT,use_shape = use_shape,config = bert_config)
  model.to(device)
  #model.load_state_dict(torch.load('/content/drive/My Drive/dl/推荐评论/CLS/best_param_score_{}'.format(i)))
  ema=torch.load('/content/drive/My Drive/dl/推荐评论/0.96840版本基础 伪标签/'+'CLS/EMA_{}'.format(i))
  ema.model = model
  ema.apply_shadow()
  pred += predict_model(model,pre_loader ,pre_shape,128, ema,config=None)

In [0]:
sub = pd.read_csv('/content/drive/My Drive/dl/推荐评论/sub.csv') 


In [0]:
sub.loc[:,'Prediction']=(pred)/8

In [29]:
sub.head()

,ID,Prediction
0,0,0.000372
1,1,0.963567
2,2,0.955745
3,3,0.977581
4,4,0.000950


In [0]:
sub.to_csv('submission_random.csv', index=False)